Link For Dataset: http://snap.stanford.edu/data/wiki-topcats.html

In [79]:
def get_files():
    #Edgelist of All Connected Nodes
    topcats = open('wiki-topcats.txt', 'r')

    #Page Names of Every Node
    topcats_page_names = open('wiki-topcats-page-names.txt', 'r')

    #Categories & All Pages Within Each Category
    topcats_categories = open('wiki-topcats-categories.txt', 'r')
    
    return topcats, topcats_page_names, topcats_categories

def load_data(topcats):
    data = np.loadtxt(topcats, dtype=int)
    return data

#Function to get all category names to a list
def all_categories(cat_list, lines):
    for line in lines:
        line = line.strip()
        line = line.split(' ')
        category = line[0]
        cat_list.append(category)
        
    return cat_list

#Function to convert categories (key) & pages in each category (value) into a dictionary
def lines_to_dict(dictionary, lines):
    for line in lines:
        line = line.strip()
        line = line.split(' ')
        key = line[0]
        values = line[1:]
        int_values = [eval(i) for i in values]
        dictionary[key] = int_values
        
    return dictionary

def make_catlist_catdict(topcats_categories):
    category_lines = topcats_categories.readlines()
    cat_list = []
    cat_list = all_categories(cat_list, category_lines)

    cat_dict = {}
    cat_dict = lines_to_dict(cat_dict, category_lines)
    
    return cat_list, cat_dict

#Only necessary if you want to see the page names/articles
def get_pages(topcats_page_names):
    page_lines = topcats_page_names.readlines()

    pages = {}

    for line in page_lines:
        line = line.strip()
        line = line.split(' ')
        key = int(line[0])
        values = line[1:]
        str_values = ' '.join(values)
        pages[key] = str_values
        
    return pages

def make_digraph(data):
    DG = nx.DiGraph()
    DG.add_edges_from(data)
    
    return DG

#Returns two random categories
def get_rand_cats(cat_list):
    cat_inds = []
    start = 0
    end = len(cat_list)

    for i in range(2):
        num = random.randint(start, end)
        cat_inds.append(num)
    
    #print(cat_inds)    
    return cat_inds

def get_rand_cat_list(num_cats, cat_list):
    cat_inds = []
    start = 0
    end = len(cat_list)

    for i in range(num_cats):
        num = random.randint(start, end)
        cat_inds.append(num)
    
    #print(cat_inds)    
    return cat_inds

#Returns Array of Page Numbers from Category Dictionary 
def get_cat_pages(category, cat_dict):
    return cat_dict.get(category)

def checks(data, cat_list, DG):
    if len(data) == 28511807:
        print('Data Transferred Correctly\n')
    else:
        print('Data Transferred Incorrectly\n')
        return False
    
    if len(cat_list) == 17364:
        print('Category List Transferred Correctly\n')
    else:
        print('Category List Transferred Incorrectly\n')
        return False
    
    if DG.number_of_nodes() == 1791489:
        print('DiGraph Made Correctly\n')
    else:
        print('DiGraph Made Incorrectly\n')
        return False
    
    print('All Check Passed')
    return True

#Path Functions

#Returns length of the shortest path between Node 1 and Node 2
def shortest_path_nodes(DG, node1, node2):
    path = nx.shortest_path(DG, node1, node2)
    return len(path)

#Returns Average Shortest Path Length of One Page from the First Category to Every Page From Another Category
def avg_shortest_path_nodes(DG, cat1_page, cat2, cat_list, cat_dict):
    paths = []
    pages_in_cat2 = cat_dict.get(cat_list[cat2])
    
    for cat2_page in pages_in_cat2:
        path = shortest_path_nodes(DG, cat1_page, cat2_page)
        paths.append(path)
    
    np_paths = np.array(paths)
    avg_path_length = np.average(np_paths)
    
    return avg_path_length

#Returns the Average Shortest Path Length Between Two Categories
def avg_shortest_path_cats(DG, cat1, cat2, cat_list, cat_dict):
    avg_paths = []
    pages_in_cat1 = cat_dict.get(cat_list[cat1])
    
    for cat1_page in pages_in_cat1:
        avg_path = avg_shortest_path_nodes(DG, cat1_page, cat2, cat_list, cat_dict)
        avg_paths.append(avg_path)
    
    np_avg_paths = np.array(avg_paths)
    avg_path_cats = np.average(np_avg_paths)
    
    return avg_path_cats

#Correlation Functions

def find_correlations(DG, cat1, sub_cat_list, cat_list, cat_dict):
    correlations = {}
    
    for cat2 in sub_cat_list:
        #Does no compare against itself
        if cat1 == cat2:
            continue
        
        #key_str = 'Category 1: ' + cat_list[cat1] + '-> Category 2: ' + cat_list[cat2]
        key_str = cat_list[cat1] + ' -> ' + cat_list[cat2]
        
        #Dictionary
        #every key is the input category, and then the category it is being compared to
        #the values is the avgerage shortest path between the two categories
        key = key_str
        values = avg_shortest_path_cats(DG, cat1, cat2, cat_list, cat_dict)
        correlations[key] = values
    
    return correlations

def cat_correlations(DG, sub_cat_list, cat_list, cat_dict):
    #array of dictionaries
    correlations = []
    
    for cat in sub_cat_list:
        correlations.append(find_correlations(DG, cat, sub_cat_list, cat_list, cat_dict))   
    
    return correlations

#Min Correlation = Largset Number, meaning largest average shortest path between two categories
#Max Correlation = Smallest Number, meaning smallest avergae shortest path between two categories

def min_max_correlation(correlations, sub_cat_list, cat_list):
    i = 0
    
    #Loops through array of dictionaries
    for correlation in correlations:
        #Initial Values
        #anything larger than 0 will replcae this number
        min_corr = 0 
        #anything smaller than 10 will replace this number, 10 chosen because the longest shortest path in the whole
        #dataset is 9
        max_corr = 10
        
        #Loops through dictionary to find min and max correlations
        for key in correlation:
            corr = correlation.get(key)
            if corr > min_corr:
                min_key = key
                min_corr = correlation.get(key)

            elif corr < max_corr:
                max_key = key
                max_corr = correlation.get(key)
            
        print(cat_list[sub_cat_list[i]])
        print(f'Minimum Correlation: {min_key}\nAverage Shortest Path Between Categories: {min_corr}')
        print(f'Maximum Correlatoin: {max_key}\nAverage Shortest Path Between Categories: {max_corr}\n')
        
        i += 1
        
def generate_sublist(subset_num, cat_list):
    sub_cat_list = get_rand_cat_list(subset_num, cat_list)
    for cat in sub_cat_list:
        print(cat, cat_list[cat])
        
    return sub_cat_list

#Examples

def print_path(DG, cat1, cat2, node1, node2, page_titles):
    path = nx.shortest_path(DG, node1, node2)
    path_str = ''
    step = 1
    print(f'\nStarting {cat1}')
    for i in path:
        page = page_titles.get(i)
        path_str = path_str + page + ' -> '
        print(f'Step: {step} - Page Title: {page}')
        step += 1
    print(f'Ending {cat2}')
    print(f'\nPath: {path_str[:-4]}')
    print(f'Path As Nodes: {path}')
    print(f'Length of Shortest Path: {len(path)}')

#Provides Base Examples of What this Project is About For Explanation Purposes
def example(DG, cat_list, cat_dict, page_titles):
    ex_cats = get_rand_cats(cat_list)
    ex_cat1 = cat_list[ex_cats[0]]
    ex_cat2 = cat_list[ex_cats[1]]
    print(f'Example Categories:\n{ex_cat1}\n{ex_cat2}')
    
    ex_cat1_pages = cat_dict.get(ex_cat1)
    ex_cat2_pages = cat_dict.get(ex_cat2)
    print(f'\nExample Category Pages:\n\n{ex_cat1}\nPages: {ex_cat1_pages}\n\n{ex_cat2}\nPages: {ex_cat2_pages}')
    
    ex_cat1_node = ex_cat1_pages[0]
    ex_cat1_node_page_title = page_titles.get(ex_cat1_node)
    ex_cat2_node = ex_cat2_pages[0]
    ex_cat2_node_page_title = page_titles.get(ex_cat2_node)
    print(f'\nExample Page/Article Titles:\n\n{ex_cat1}\nPage Node: {ex_cat1_node}\nPage Title: {ex_cat1_node_page_title}\n\n{ex_cat2}\nPage Node: {ex_cat2_node}\nPage Title: {ex_cat2_node_page_title}')
    
    print('\nExample Shortest Path:')
    print_path(DG, ex_cat1, ex_cat2, ex_cat1_node, ex_cat2_node, page_titles)
    ex_cat2_node2 = ex_cat2_pages[1]
    print_path(DG, ex_cat1, ex_cat2, ex_cat1_node, ex_cat2_node2, page_titles)
    
    print(f'\nExample Category List:')
    sublist_len5 = generate_sublist(5, cat_list)
    
def insertion_sort(cat_list, cat_dict):
    sorted_cat_list = cat_list
    for i in range(1, len(sorted_cat_list)):
 
        #String of category name
        key = sorted_cat_list[i]
        
        #Number of Articles in Category
        num_articles = len(cat_dict.get(key))

        j = i-1
        while j >= 0 and num_articles < len(cat_dict.get(sorted_cat_list[j])) :
                sorted_cat_list[j + 1] = sorted_cat_list[j]
                j -= 1
        sorted_cat_list[j + 1] = key
    
    return sorted_cat_list

#Returns new dictionary of sampled categories
# sample_size = number of articles to sample from category
# cats_to_sample = list of categories to sample from (will use most_popular_cats array)
# temp_cat_list = category list to append with new category titles
# cat_dict = categoy dictionary to add new categories and sampled articles
# returns nothing
#Doing it this way for purpose of using previously made functions, cat_correlations takes 
#numbers to represent the index of the category in cat_list. Allows for using same logic.

def sample_category(sample_size, cats_to_sample, temp_cat_list, cat_dict):
    for cat in cats_to_sample:
        #String to use as new key in dictionary and category array
        sample_cat_name = cat + '-sample'
        #Appending category array with new category
        temp_cat_list.append(sample_cat_name)
        
        #Empty array for appending randomly selected articles
        sampled_articles = []
        
        #Gets the value/array of articles from dictionary
        articles_in_cat = cat_dict.get(cat)
        
        #Start and Ending indexes that is randomly selected from article array
        start_ind = 0
        end_ind = len(articles_in_cat)-1
        
        #Loops for number of samples required
        for i in range(sample_size):
            #Loops until article not previously sampled is chosen, prevents potential repeats in the sample 
            while True:
                #random index
                rand_ind = random.randint(start_ind, end_ind)

                #gets value/article at random index
                article = articles_in_cat[rand_ind]
                
                if article not in sampled_articles:
                    sampled_articles.append(article)
                    break
            
        #Put Sampled Articles into Dictionary
        cat_dict[sample_cat_name] = sampled_articles

In [3]:
#Start Up - will take a few minutes

#Imports
import pandas as pd
import numpy as np
import networkx as nx
import random

#Files
topcats, topcats_page_names, topcats_categories = get_files()
#Page Titles
page_titles = get_pages(topcats_page_names) 
#Data/Edge List
data = load_data(topcats)
#Category Name List & Category Page Dictionary
cat_list, cat_dict = make_catlist_catdict(topcats_categories)
#Make Graph
DG = make_digraph(data)
#Checks
checks(data, cat_list, DG)

Data Transferred Correctly

Category List Transferred Correctly

DiGraph Made Correctly

All Check Passed


True

In [6]:
temp_cat_list = cat_list.copy()
sorted_cat_list = insertion_sort(temp_cat_list, cat_dict)

In [7]:
most_popular_cats = sorted_cat_list[-5:]
most_popular_cats

['Category:American_film_actors;',
 'Category:American_films;',
 'Category:English-language_films;',
 'Category:Year_of_birth_missing_(living_people);',
 'Category:Living_people;']

In [8]:
for i in most_popular_cats:
    print(i, len(cat_dict.get(i)))

Category:American_film_actors; 13938
Category:American_films; 15302
Category:English-language_films; 22699
Category:Year_of_birth_missing_(living_people); 34721
Category:Living_people; 418223


In [45]:
sublist_len5 = generate_sublist(5, cat_list)
for i in sublist_len5:
    print(i, len(cat_dict.get(cat_list[i])))

8074 Category:Investment_banks;
10333 Category:Actors_from_Paris;
5930 Category:American_heavy_metal_singers;
14823 Category:Business_law;
11479 Category:16th-century_Italian_people;
8074 135
10333 251
5930 275
14823 128
11479 260


In [46]:
most_popular_cats_and_random_cats = most_popular_cats.copy()
# most_popular_cats_and_random_cats.append(cat_list[11306])
# most_popular_cats_and_random_cats.append(cat_list[16059])
# most_popular_cats_and_random_cats.append(cat_list[618])
# most_popular_cats_and_random_cats.append(cat_list[16312])
# most_popular_cats_and_random_cats.append(cat_list[12291])
most_popular_cats_and_random_cats.append(cat_list[8074])
most_popular_cats_and_random_cats.append(cat_list[10333])
most_popular_cats_and_random_cats.append(cat_list[5930])
most_popular_cats_and_random_cats.append(cat_list[14823])
most_popular_cats_and_random_cats.append(cat_list[11479])

In [80]:
temp_cat_dict = {}
temp_arr = []
sample_size = 100

In [18]:
for i in most_popular_cats:
    temp_arr.append(i)
    values = cat_dict.get(i)
    temp_cat_dict[i] = values

In [81]:
for i in most_popular_cats_and_random_cats:
    temp_arr.append(i)
    values = cat_dict.get(i)
    temp_cat_dict[i] = values

In [82]:
temp_arr

['Category:American_film_actors;',
 'Category:American_films;',
 'Category:English-language_films;',
 'Category:Year_of_birth_missing_(living_people);',
 'Category:Living_people;',
 'Category:Investment_banks;',
 'Category:Actors_from_Paris;',
 'Category:American_heavy_metal_singers;',
 'Category:Business_law;',
 'Category:16th-century_Italian_people;']

In [20]:
for i in most_popular_cats:
    print(i, len(temp_cat_dict.get(i)))

Category:American_film_actors; 13938
Category:American_films; 15302
Category:English-language_films; 22699
Category:Year_of_birth_missing_(living_people); 34721
Category:Living_people; 418223


In [83]:
for i in most_popular_cats_and_random_cats:
    print(i, len(temp_cat_dict.get(i)))
print(len(temp_arr))

Category:American_film_actors; 13938
Category:American_films; 15302
Category:English-language_films; 22699
Category:Year_of_birth_missing_(living_people); 34721
Category:Living_people; 418223
Category:Investment_banks; 135
Category:Actors_from_Paris; 251
Category:American_heavy_metal_singers; 275
Category:Business_law; 128
Category:16th-century_Italian_people; 260
10


In [53]:
#sample_category(sample_size, most_popular_cats, temp_arr, temp_cat_dict)

In [84]:
sample_category(sample_size, most_popular_cats_and_random_cats, temp_arr, temp_cat_dict)

In [85]:
temp_arr

['Category:American_film_actors;',
 'Category:American_films;',
 'Category:English-language_films;',
 'Category:Year_of_birth_missing_(living_people);',
 'Category:Living_people;',
 'Category:Investment_banks;',
 'Category:Actors_from_Paris;',
 'Category:American_heavy_metal_singers;',
 'Category:Business_law;',
 'Category:16th-century_Italian_people;',
 'Category:American_film_actors;-sample',
 'Category:American_films;-sample',
 'Category:English-language_films;-sample',
 'Category:Year_of_birth_missing_(living_people);-sample',
 'Category:Living_people;-sample',
 'Category:Investment_banks;-sample',
 'Category:Actors_from_Paris;-sample',
 'Category:American_heavy_metal_singers;-sample',
 'Category:Business_law;-sample',
 'Category:16th-century_Italian_people;-sample']

In [86]:
len(temp_arr)

20

In [16]:
len(temp_cat_dict.get(temp_arr[5]))

100

In [57]:
sub_cat_list = [5, 6, 7, 8, 9]
sub_cat_list

[5, 6, 7, 8, 9]

In [89]:
sub_cat_list = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
sub_cat_list

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [58]:
test_sample_correlations = cat_correlations(DG, sub_cat_list, temp_arr, temp_cat_dict)
test_sample_correlations

[{'Category:American_film_actors;-sample -> Category:American_films;-sample': 5.0309,
  'Category:American_film_actors;-sample -> Category:English-language_films;-sample': 4.8461,
  'Category:American_film_actors;-sample -> Category:Year_of_birth_missing_(living_people);-sample': 5.7573,
  'Category:American_film_actors;-sample -> Category:Living_people;-sample': 5.880499999999999},
 {'Category:American_films;-sample -> Category:American_film_actors;-sample': 4.979,
  'Category:American_films;-sample -> Category:English-language_films;-sample': 4.9096,
  'Category:American_films;-sample -> Category:Year_of_birth_missing_(living_people);-sample': 5.949599999999999,
  'Category:American_films;-sample -> Category:Living_people;-sample': 6.077799999999999},
 {'Category:English-language_films;-sample -> Category:American_film_actors;-sample': 4.9158,
  'Category:English-language_films;-sample -> Category:American_films;-sample': 5.047000000000001,
  'Category:English-language_films;-sample 

In [90]:
test_sample_correlations = cat_correlations(DG, sub_cat_list, temp_arr, temp_cat_dict)
test_sample_correlations

[{'Category:American_film_actors;-sample -> Category:American_films;-sample': 4.970899999999999,
  'Category:American_film_actors;-sample -> Category:English-language_films;-sample': 4.9525,
  'Category:American_film_actors;-sample -> Category:Year_of_birth_missing_(living_people);-sample': 5.803299999999999,
  'Category:American_film_actors;-sample -> Category:Living_people;-sample': 5.869300000000001,
  'Category:American_film_actors;-sample -> Category:Investment_banks;-sample': 5.486400000000001,
  'Category:American_film_actors;-sample -> Category:Actors_from_Paris;-sample': 5.0421000000000005,
  'Category:American_film_actors;-sample -> Category:American_heavy_metal_singers;-sample': 5.3364,
  'Category:American_film_actors;-sample -> Category:Business_law;-sample': 5.1436,
  'Category:American_film_actors;-sample -> Category:16th-century_Italian_people;-sample': 5.706200000000001},
 {'Category:American_films;-sample -> Category:American_film_actors;-sample': 4.895300000000001,
 

In [61]:
min_max_correlation(test_sample_correlations, sub_cat_list, temp_arr)

Category:American_film_actors;-sample
Minimum Correlation: Category:American_film_actors;-sample -> Category:Living_people;-sample
Average Shortest Path Between Categories: 5.880499999999999
Maximum Correlatoin: Category:American_film_actors;-sample -> Category:English-language_films;-sample
Average Shortest Path Between Categories: 4.8461

Category:American_films;-sample
Minimum Correlation: Category:American_films;-sample -> Category:Living_people;-sample
Average Shortest Path Between Categories: 6.077799999999999
Maximum Correlatoin: Category:American_films;-sample -> Category:English-language_films;-sample
Average Shortest Path Between Categories: 4.9096

Category:English-language_films;-sample
Minimum Correlation: Category:English-language_films;-sample -> Category:Living_people;-sample
Average Shortest Path Between Categories: 5.988
Maximum Correlatoin: Category:American_films;-sample -> Category:English-language_films;-sample
Average Shortest Path Between Categories: 10

Categor

In [91]:
min_max_correlation(test_sample_correlations, sub_cat_list, temp_arr)

Category:American_film_actors;-sample
Minimum Correlation: Category:American_film_actors;-sample -> Category:Living_people;-sample
Average Shortest Path Between Categories: 5.869300000000001
Maximum Correlatoin: Category:American_film_actors;-sample -> Category:English-language_films;-sample
Average Shortest Path Between Categories: 4.9525

Category:American_films;-sample
Minimum Correlation: Category:American_films;-sample -> Category:Living_people;-sample
Average Shortest Path Between Categories: 5.9615
Maximum Correlatoin: Category:American_films;-sample -> Category:Actors_from_Paris;-sample
Average Shortest Path Between Categories: 5.0261000000000005

Category:English-language_films;-sample
Minimum Correlation: Category:English-language_films;-sample -> Category:Living_people;-sample
Average Shortest Path Between Categories: 5.9620999999999995
Maximum Correlatoin: Category:English-language_films;-sample -> Category:Actors_from_Paris;-sample
Average Shortest Path Between Categories:

In [92]:
temp_cat_dict.get(temp_arr[13])

[944217,
 751875,
 590166,
 570236,
 1638092,
 467909,
 443203,
 423316,
 76851,
 909843,
 628227,
 243454,
 144975,
 1324692,
 1621295,
 1718334,
 319913,
 1602001,
 733638,
 1604255,
 1030656,
 1166431,
 629728,
 724125,
 632798,
 1722080,
 1782332,
 110651,
 270151,
 1129958,
 1591617,
 821490,
 400549,
 601920,
 909500,
 570426,
 1203608,
 749374,
 551760,
 1694543,
 806467,
 551439,
 1294474,
 352668,
 449467,
 960228,
 1700605,
 1014031,
 588502,
 1150212,
 1239041,
 601197,
 155275,
 1162031,
 68835,
 1554930,
 1262331,
 815237,
 451191,
 459978,
 501211,
 825928,
 136898,
 652361,
 1534522,
 438329,
 1468162,
 637599,
 560973,
 1583408,
 604535,
 533227,
 1738951,
 1680791,
 1689455,
 1255389,
 632629,
 343907,
 1298854,
 1378280,
 1612073,
 1365965,
 1112674,
 1763575,
 922216,
 629667,
 211645,
 1173309,
 457849,
 922152,
 1001433,
 687380,
 1775387,
 1125710,
 1550206,
 1013746,
 744651,
 1322934,
 804048,
 688709]

In [97]:
print(page_titles.get(944217))
print(page_titles.get(751875))
print(page_titles.get(590166))
print(page_titles.get(570236))

Edna Gundersen
Michael de Koningh
Faryal
Jean Engler
